<a href="https://colab.research.google.com/github/Shubhankar10/KDAG-Model/blob/main/KGP1CSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**KDAG**   *Hackathon Tasks*


> Shubhankar Tiwary

> Saurabh Yadav 

> Chaitanya Rao






Predict Best channel for physicians unsing given data

### ***Get Data***

In [ ]:
!pip install kaggle

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
id = "1geQKwlJDGB9WiKK8YR6H2-8HFDhlvdZX"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('kaggle.json')

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download -c kdsh-data-solution-task-1

In [ ]:
! unzip kdsh-data-solution-task-1.zip

In [ ]:
import json
import numpy as np
import pandas as pd
import datetime
from prophet import Prophet
import matplotlib.pyplot as plt

Read Data

In [ ]:
hcp = pd.read_excel('/content/HCP_Data_KDAG_Hackathon/HCP_Data_KDAG_Hackathon.xlsx')
# patient = pd.read_excel('/content/Patient_Data_KDAG_Hackathon/Patient_Data_KDAG_Hackathon.xlsx')
# print(hcp.head())
# print(patient.head())

In [ ]:
print('Length of HCP: ',len(hcp))

### **DataFrame**


In [ ]:
data = pd.DataFrame({
  'Physician_ID' : hcp['Physician_ID'],
  'Brand_Rx' : hcp['Brand_Rx'],
  'Sales_Rep_Calls' : hcp['Sales_Rep_Calls'],
  'Samples_Dropped' : hcp['Samples_Dropped'],
  'Emails_Delivered' : hcp['Emails_Delivered'],
  'Time_Period' : hcp['Time_Period'],
  'Physician_Segment' : hcp['Physician_Segment'],
})


In [ ]:
''' Change Non Zero to One '''

# where mai False hai toh value swap hogi, .mask mai true hai toh swap hogi
data['Sales_Rep_Calls'] = data['Sales_Rep_Calls'].where(data['Sales_Rep_Calls']<=0,1)
data['Samples_Dropped'] = data['Samples_Dropped'].where(data['Samples_Dropped']<=0,1)
data['Emails_Delivered'] = data['Emails_Delivered'].where(data['Emails_Delivered']<=0,1)


In [ ]:
''' Replace Physician Segment '''

data = data.replace('3-Low',3)
data = data.replace('2-Medium',2)
data = data.replace('1-High',1)
data['Physician_Segment'] = data['Physician_Segment'].apply(pd.to_numeric)
data.sort_values(by=['Time_Period'])

In [ ]:
''' Convert Date Format '''

data['Time_Period'] = pd.to_datetime(data['Time_Period'], format='%Y%m%d')

In [ ]:
''' Multiply BrandRX '''

model_data = pd.DataFrame({
  'Physician_ID' : data['Physician_ID'],
  'Time_Period' : data['Time_Period'],
  'Brand_Rx' : data['Brand_Rx'],
  'Sales_Rep_Calls' : data['Sales_Rep_Calls'] * data['Brand_Rx'],
  'Samples_Dropped' : data['Samples_Dropped'] * data['Brand_Rx'],
  'Emails_Delivered' : data['Emails_Delivered'] * data['Brand_Rx'],
})


### **Model**

In [ ]:
# Ek baar he run karna iss block ko bas 
final = pd.DataFrame()
Physicians = data["Physician_ID"].unique()
Physicians = Physicians[:len(Physicians)//50] # Taking less for trial
print(len(Physicians))

In [ ]:
def Model(df):
  model = Prophet(weekly_seasonality=True)
  model.fit(df)

  future = model.make_future_dataframe(periods=7)
  future.tail()

  forecast = model.predict(future)
  forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

  output = forecast['yhat'].mean()
  print('Output : ', forecast['yhat'].mean() )

  # fig1 = model.plot(forecast)
  # fig2 = model.plot_components(forecast)

  return output

In [ ]:
# P = ['axt00001']

# for i in Physicians :

#   print(i)
#   individual_data = model_data.loc[(data["Physician_ID"] == i)]

#   call_df = pd.DataFrame({
#     'Time_Period' : individual_data['Time_Period'],
#     'Sales_Rep_Calls' : individual_data['Sales_Rep_Calls'],
#   })
#   sample_df = pd.DataFrame({
#     'Time_Period' : individual_data['Time_Period'],
#     'Samples_Dropped' : individual_data['Samples_Dropped'],
#   })
#   email_df = pd.DataFrame({
#     'Time_Period' : individual_data['Time_Period'],
#     'Emails_Delivered' : individual_data['Emails_Delivered'],
#   })

#   call_df.columns = ['ds','y']
#   sample_df.columns = ['ds','y']
#   email_df.columns = ['ds','y']

#   call = Model(call_df)
#   sample = Model(sample_df)
#   email = Model(email_df)

#   L=[]
#   L.append(call)
#   L.append(sample)
#   L.append(email)
#   L.sort()
#   print(L)
#   # call_df.plot(x='ds')
#   # email_df.plot(x='ds')
#   # sample_df.plot(x='ds')
#   # plt.show()

#   # Call -> 0
#   # Sample -> 1
#   # Email -> 2

#   # Condition for equal left
#   if call > email and call  > sample :
#     if email > sample :
#       second_best = 2
#     else :
#       second_best =1
#     best = 0
#   elif sample > email :
#     if email > call :
#       second_best = 2
#     else :
#       second_best = 0
#     best = 1
#   else :
#     if call > sample :
#       second_best = 0
#     else :
#       second_best = 1
#     best = 2

#   fin = pd.DataFrame({
#       'HCP' : i,
#       'Call' :  call,
#       'Sample' : sample,
#       'Email' : email,
#       'Best_ch' : best,
#       'SecondBest' : second_best,
#       'Max' : L[2],
#       'SecondMax' : L[1],
#   }, index=[0])

#   print(fin)

#   final = final.append(fin)


In [ ]:
# Call -> 0
# Sample -> 1
# Email -> 2
final

""


### **Final Dataset**

In [ ]:
''' import trained data '''
from google.colab import files

id = "1zpNZg4gJR2i5kqzDqaTDCnHjZerf3K8k"
finalDrive = drive.CreateFile({'id':id})
finalDrive.GetContentFile('Final.csv')

In [ ]:
final = pd.read_csv('/content/Final.csv')
final.sample(5)

In [ ]:
final['SecondMax'] = final[["Call", "Sample", "Email"]].apply(lambda row: row.nlargest(2).iat[-1], axis=1)
final['ThirdMax'] = final[["Call", "Sample", "Email"]].apply(lambda row: row.nlargest(3).iat[-1], axis=1)
final.tail()

In [ ]:
t = {
    'Call' : 0,
    'Sample' : 1,
    'Email' : 2,
}

In [ ]:
final['SecondBest'] = final[["Call", "Sample", "Email"]].apply(lambda row: t[row.nlargest(2).index[-1]], axis=1)
final['ThirdBest'] = final[["Call", "Sample", "Email"]].apply(lambda row: t[row.nlargest(3).index[-1]], axis=1)
final.sample(7)

In [ ]:
submit = pd.DataFrame({'Physician_ID' : final['HCP'] , 'Channel' : final['Best_ch'],})
submitTask2 = pd.DataFrame({'Physician_ID' : final['HCP'] , 'Expected_TRx' : final['Max'],})

## **Output**

In [ ]:
# Constraint 1:
# Pure data ka calls uss k segment se zayada nai hona chahiye
# Given data se calculate karlo phir predicted mai se jaha jaha 0 hai unn k liye check karo ki
#   add karne k baad segment se zyada toh nai hora

# Constraint 2:
# Predicted data mai se sirf 25% hcp ka best channel Sample hone chahiye
# Toh total nikalne k baad jitne jagah 1 hai uska count leke check karna hai

# Constraint 3:
# Jan last week ka sum BrandRx nikal k predicted vale se compare karo kisi ka bhi +-10% se zayada
#   nai hona chahiye teeno Prediction k liye

### **Contraint 3**

In [ ]:
filtered_df = data.loc[(data['Time_Period'] >= pd.Timestamp(2020,1,24))]
F = filtered_df.groupby(['Physician_ID','Physician_Segment','Time_Period'], as_index=False).sum()
F = filtered_df.groupby(['Physician_ID','Physician_Segment'], as_index=False).mean()
F

In [ ]:
def constraint_3(x, s, stop):
  pred_brand = x[s]
  og_brand = F.loc[F['Physician_ID'].str.contains(x['HCP'])]['Brand_Rx'].iat[-1]
  high = og_brand + 0.1 * og_brand
  low = og_brand - 0.1 * og_brand
  x['Og'] = og_brand
  x['high'] = high
  x['low'] = low
  if low <= pred_brand <= high:
    x['out'] = 'Yes'
    if s == 'Max':
      i = submit.loc[submit['Physician_ID'].str.contains(x['HCP'])].index
      submit.iat[i[0],1] = x['Best_ch']

      j = submitTask2.loc[submitTask2['Physician_ID'].str.contains(x['HCP'])].index
      submitTask2.iat[i[0],1] = x['Max']

      # submit.loc[submit['Physician_ID'].str.contains(x['HCP'])]['Channel'] = x['Best_ch']
      # submitTask2.loc[submitTask2['Physician_ID'].str.contains(x['HCP'])]['Expected_TRx'] = x['Max']
    elif s== 'SecondMax':
      i = submit.loc[submit['Physician_ID'].str.contains(x['HCP'])].index
      submit.iat[i[0],1] = x['SecondBest']

      j = submitTask2.loc[submitTask2['Physician_ID'].str.contains(x['HCP'])].index
      submitTask2.iat[i[0],1] = x['SecondMax']

      # submit.loc[submit['Physician_ID'].str.contains(x['HCP'])]['Channel'] = x['SecondBest']
      # submitTask2.loc[submitTask2['Physician_ID'].str.contains(x['HCP'])]['Expected_TRx'] = x['SecondMax']
    else:
      i = submit.loc[submit['Physician_ID'].str.contains(x['HCP'])].index
      submit.iat[i[0],1] = x['ThirdBest']

      j = submitTask2.loc[submitTask2['Physician_ID'].str.contains(x['HCP'])].index
      submitTask2.iat[i[0],1] = x['ThirdMax']
      # submit.loc[submit['Physician_ID'].str.contains(x['HCP'])]['Channel'] = x['ThirdBest']
      # submitTask2.loc[submitTask2['Physician_ID'].str.contains(x['HCP'])]['Expected_TRx'] = x['ThirdMax']

  else:
    if(s == 'Max'):
      return constraint_3(x,'SecondMax', True)
    elif s == 'SecondMax':
      return constraint_3(x,'ThirdMax', True)
    else:
      x['out'] = 'No'
      return x
  return x
  

In [ ]:
constraint3 = final.apply(lambda x: constraint_3(x, 'Max', False), axis=1)

### **Constraint 1**


In [ ]:
''' For without importing '''
# call_list = final.loc[(final["Best_ch"] == 0 )]
# HCP = list(call_list['HCP'])
# D = data.groupby(['Physician_ID'])

# for i in range(len(HCP)):

#   sbc = final['SecondBest'].values[i] #Second Best Channel

#   individual = D.get_group(str(HCP[i]))
#   sum = individual['Sales_Rep_Calls'].sum()
#   segment = int(individual['Physician_Segment'].unique())

#   #Change values for those aplicable to the second best channel
#   if segment == 3 and sum >= 12 :
#     channel = sbc
#   elif segment == 2 and sum >= 24 :
#     channel = sbc
#   elif segment == 1 and sum >= 48 :
#     channel = sbc
#   else:
#     channel = 0
#   temp = pd.DataFrame({'Physician_ID' : HCP[i] , 'Channel' : channel,},index=[0])
#   submit = submit.append(temp)

# print(submit)

In [ ]:
D = pd.DataFrame(data.groupby(['Physician_ID','Physician_Segment'], as_index = False).sum())
D.head()

In [ ]:
def constraint1(x):
  x['Out'] = 'Yes'
  if x['Best_ch'] == 0:
    og_calls = D.loc[D['Physician_ID'] == x['HCP']]['Sales_Rep_Calls'].iat[-1]
    total = og_calls + 1
    segment = D.loc[D['Physician_ID'] == x['HCP']]['Physician_Segment'].iat[-1]

    if (segment == 3 and total >= 12) or (segment == 2 and total >= 24) or (segment == 1 and total >= 48):
      i = submit.loc[submit['Physician_ID'].str.contains(x['HCP'])].index
      submit.iat[i[0],1] = x['SecondBest']
      j = submitTask2.loc[submitTask2['Physician_ID'].str.contains(x['HCP'])].index
      submitTask2.iat[i[0],1] = x['SecondMax']
      # submitTask2.loc[submitTask2['Physician_ID'].str.contains(x['HCP'])]['Expected_TRx'] = x['SecondMax']
      x['Out'] = 'No'
    else:
      x['Out'] = 'Yes'
  return x

In [ ]:
constraint1 = final.apply(lambda x: constraint1(x), axis=1)

In [ ]:
# call_list['Max'] = call_list[['Max', 'Sum','SecondMax']].apply(lambda x: x['SecondMax'] if x['Sum'] < 0 else x['Max'], axis=1)
# call_list['Best_ch'] = call_list[['Best_ch', 'Sum','SecondBest']].apply(lambda x: x['SecondBest'] if x['Sum'] < 0 else x['Best_ch'], axis=1)
# call_list.head()

### **Constraint 2** 

In [ ]:
samples = final.loc[(final["Best_ch"] == 1 )]
count = samples['Best_ch'].count()
count
# HCP = list(samples['HCP'])
# flag = 0

# print('Total HCP for Samples : ' , count)
# if count > (len(Physicians)//25):
#   flag = 1

# # Change values of those which are more than 25% to 2 ie email 
# if flag == 1 :
#   for i in range((len(Physicians)//25)-1,len(Physicians)): 
#     samples['Best_ch'] = 2

# temp = pd.DataFrame({'Physician_ID' : HCP , 'Channel' : samples['Best_ch'],})

# submit = submit.append(temp)
# print(submit)

74

## **Submit**

In [ ]:
submitTask2 = pd.DataFrame({'Physician_ID' : final['HCP'] , 'Expected_TRx' : final['Max'],})

In [ ]:
submit

In [ ]:
submitTask2

In [ ]:
submit.to_csv('submit.csv', encoding = 'utf-8-sig',index=False) 
files.download('submit.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
submitTask2.to_csv('submitTask2.csv', encoding = 'utf-8-sig',index=False) 
files.download('submitTask2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>